<a href="https://colab.research.google.com/github/maitri2905/NaiveBayesClassifier/blob/main/Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name : Maitri Nilesh Patel</br>
UTA ID: 1001716017</br>
Assignment 03 </br>


# Importing Dataset for Movie Review </br>
***Here I have imported yelp Dataset from Kaggle Sentiment labelled Dataset</br>***


In [1169]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1170]:
import pandas as pd
import matplotlib.pyplot as plt

Data = pd.read_csv('/content/gdrive/My Drive/Dataset/yelp_labelled.txt', delimiter='\t', header=None, names= ['Reviews', 'Sentiments'])
Data.head()

,Reviews,Sentiments
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [1171]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Reviews     1000 non-null   object
 1   Sentiments  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [1172]:
Data.isnull().sum()

Reviews       0
Sentiments    0
dtype: int64

In [1173]:
Data = pd.DataFrame(Data)

In [1174]:
import os
import numpy as np
from pandas import Series
import re 
from sys import path
from collections import Counter

In [1175]:
Data.Sentiments.value_counts()

1    500
0    500
Name: Sentiments, dtype: int64

# **A) Divide the Dataset as Training and Development Data.</br>**
# #Here I have divided data in the ratio of 76:24 as Train and Development.</br>


In [1176]:
shuffle = Data.sample(frac=1).to_numpy()
Datasize = int(0.76 * len(Data))
Training_Dataset = shuffle[:Datasize]
Dev_Dataset = shuffle[Datasize:]
print(Training_Dataset)
print(Dev_Dataset)

[["I don't think I'll be running back to Carly's anytime soon for food."
  0]
 ['Great food.' 1]
 ['the spaghetti is nothing special whatsoever.' 0]
 ...
 ['The waitresses are very friendly.' 1]
 ["I mean really, how do you get so famous for your fish and chips when it's so terrible!?!"
  0]
 ['Both of the egg rolls were fantastic.' 1]]
[['Went in for happy hour, great list of wines.' 1]
 ['We were promptly greeted and seated.' 1]
 ['The Heart Attack Grill in downtown Vegas is an absolutely flat-lined excuse for a restaurant.'
  0]
 ['I dont think I will be back for a very long time.' 0]
 ['Not tasty and the texture was just nasty.' 0]
 ['The real disappointment was our waiter.' 0]
 ['Sooooo good!!' 1]
 ['I got home to see the driest damn wings ever!' 0]
 ['Weird vibe from owners.' 0]
 ['Will not be back.' 0]
 ['The staff is always super friendly and helpful, which is especially cool when you bring two small boys and a baby!'
  1]
 ['Food quality has been horrible.' 0]
 ['Service was e

In [1177]:
print("Shape of Training_Dataset",{Training_Dataset.shape})
print("Shape of Dev_Dataset",{Dev_Dataset.shape})

Shape of Training_Dataset {(760, 2)}
Shape of Dev_Dataset {(240, 2)}


In [1178]:
x_Train = Training_Dataset[:,0]
y_Train = Training_Dataset[:,-1]

x_Dev = Dev_Dataset[:,0]
y_Dev = Dev_Dataset[:,-1]



# **B) Building a Vocabulary list as below**

In [1179]:
V = {}
V_List = []
def build_V_List(Data):
  V = counting_number_of_unique_words(Data)
  for i in V.keys():
    V_List.append(i)
    return V_List

In [1180]:
Count = {}
def counting_number_of_unique_words(Data):
  for L in Data:
    for W in L.split():
      if W in Count:
        Count[W] = Count[W] + 1
      else:
          Count[W] = 1     
  return Count



In [1181]:
Count = counting_number_of_unique_words(x_Train)
print("Unique words in Dataset before Removing less Occuring Words : ", len(Count))

Unique words in Dataset before Removing less Occuring Words :  2471


In [1182]:
print (Count)

{'I': 219, "don't": 19, 'think': 10, "I'll": 6, 'be': 49, 'running': 4, 'back': 26, 'to': 171, "Carly's": 1, 'anytime': 4, 'soon': 3, 'for': 72, 'food.': 13, 'Great': 15, 'the': 322, 'spaghetti': 1, 'is': 128, 'nothing': 5, 'special': 2, 'whatsoever.': 1, 'The': 133, 'chicken': 12, 'dishes': 7, 'are': 51, 'OK,': 1, 'beef': 4, 'like': 33, 'shoe': 1, 'leather.': 1, "won't": 13, 'try': 5, 'going': 12, 'there': 17, 'even': 13, 'if': 13, "it's": 10, 'empty.': 1, 'However,': 2, 'my': 38, 'recent': 1, 'experience': 5, 'at': 40, 'this': 77, 'particular': 1, 'location': 3, 'was': 223, 'not': 77, 'so': 52, 'good.': 16, "Don't": 3, 'waste': 4, 'your': 15, 'time': 25, 'here.': 10, 'Once': 1, 'food': 56, 'arrives': 1, 'meh.': 1, 'Now': 3, 'am': 6, 'getting': 7, 'angry': 1, 'and': 281, 'want': 11, 'damn': 1, 'pho.': 1, 'WAAAAAAyyyyyyyyyy': 1, 'over': 6, 'rated': 1, 'all': 22, 'saying.': 1, 'Please': 1, 'stay': 2, 'away': 4, 'from': 16, 'shrimp': 3, 'stir': 1, 'fried': 4, 'noodles.': 1, 'i': 8, 'felt

# Removing the repeated words

In [1183]:
Repeat_W = []
for i in Count.keys():
  if (Count[i]<7):
    Repeat_W.append(i)
for i in Repeat_W:
  del Count[i]
len (Count)

181

# Removing the less Occuring Words and Cleaning the Data

In [1184]:
def Remove_less_occuring_word(Data, Count):
  for i, Reviews in enumerate(Data):
    Cleaning = []
    for W in Reviews.split():
      if W in Count.keys():
         Cleaning.append(W)
    Cleaning = ' '.join(Cleaning)
    Data[i] = Cleaning
  return Data

In [1185]:
x_Train = Remove_less_occuring_word(x_Train, Count)
print("Total unique words after Removing less occuring Words : ", len(x_Train))
print(Count)
print('')

Total unique words after Removing less occuring Words :  760
{'I': 219, "don't": 19, 'think': 10, 'be': 49, 'back': 26, 'to': 171, 'for': 72, 'food.': 13, 'Great': 15, 'the': 322, 'is': 128, 'The': 133, 'chicken': 12, 'dishes': 7, 'are': 51, 'like': 33, "won't": 13, 'going': 12, 'there': 17, 'even': 13, 'if': 13, "it's": 10, 'my': 38, 'at': 40, 'this': 77, 'was': 223, 'not': 77, 'so': 52, 'good.': 16, 'your': 15, 'time': 25, 'here.': 10, 'food': 56, 'getting': 7, 'and': 281, 'want': 11, 'all': 22, 'from': 16, 'i': 8, 'how': 9, 'could': 11, 'you': 41, 'being': 14, 'Best': 7, 'in': 77, 'our': 27, 'came': 13, 'This': 34, 'will': 19, 'recommend': 9, 'of': 100, 'it': 64, 'were': 49, 'still': 7, 'quite': 7, 'them': 10, 'out': 14, 'who': 7, 'can': 14, 'come': 9, 'they': 28, 'a': 168, 'but': 46, 'here': 28, 'know': 8, 'what': 12, 'about': 13, 'pizza': 8, 'good': 43, 'very': 47, 'place': 55, 'me': 7, 'little': 9, 'with': 55, 'we': 37, 'We': 27, 'ordered': 9, 'on': 39, 'nice': 14, 'had': 50, 'mu

In [1186]:
Reviews_0_Train = Training_Dataset[y_Train == 0]
Reviews_1_Train = Training_Dataset[y_Train == 1]

In [1187]:
print("Negative Reviews in Training Dataset Count : ", len((Reviews_0_Train)))

Negative Reviews in Training Dataset Count :  373


In [1188]:
print("Positive Reviews in Training Dataset Count : ", len((Reviews_1_Train)))

Positive Reviews in Training Dataset Count :  387


# **Making Directory**

In [1189]:
def Total_Words(Data, Count):
  New_Directory = {}
  for W in Count.keys():
    Count1 = 0
    for Reviews in Data:
      if W in Reviews:
        Count1 = Count1 + 1
    New_Directory [W] = Count1
  return New_Directory

In [1190]:
Occur_Word = Total_Words(x_Train, Count)
print("The count in dataset", Occur_Word['the'])

The count in dataset 298


# **C) Finding Basic Probabilities**

In [1191]:
def basic_probability(y_Train):
  Probability = {}
  for a in np.unique(y_Train):
    Count1 = sum(y_Train == a)
    Probability[a] = Count1 / y_Train.size
  return Probability

In [1192]:
Probability = basic_probability(y_Train)
print("Negative probabiity",Probability[0])
print("Positive probability",Probability[1])

Negative probabiity 0.49078947368421055
Positive probability 0.5092105263157894


**# Probability of Occurance of word "the"**

In [1193]:
def Probability_of_the_in_Dataset (Data,W):
  Count1 = 0
  if W not in Occur_Word.keys():
    Count1 = 0
  else:
    Count1 = Occur_Word[W]
  return Count1 / len(Data)


In [1194]:
The_Probability = Probability_of_the_in_Dataset(x_Train,'the')

In [1195]:
print("Probability of the in dataset P[the]:",The_Probability * 100)

Probability of the in dataset P[the]: 39.21052631578947


# **Conditional Probability based on the Sentiments**

In [1196]:
def Positive_Words (Data, label, Count):
  P_Words = {}
  for W in Count.keys():
    P = 0
    for index, Reviews in enumerate(Data):
      if W in Reviews.split() and label[index] == 1:
        P = P + 1
    P_Words[W] = P
  return P_Words

In [1197]:
def Negative_Words (Data, label, Count):
 N_Words = {}
 for W in Count.keys():
    N = 0
    for index, Reviews in enumerate(Data):
      if W in Reviews.split() and label[index] == 0:
        N = N + 1
    N_Words[W] = N
 return N_Words

In [1198]:
P_Words = Positive_Words(x_Train, y_Train, Count)
N_Words = Negative_Words(x_Train, y_Train, Count)

In [1199]:
P_Words['the']

134

In [1200]:
N_Words['the']

124

In [1201]:
def docu_positive (Data):
  P_doc = 0
  for i in range(len(Data)):
    if (Data[i] == 1):
      P_doc = P_doc + 1
  return P_doc
  

In [1202]:
def docu_negative (Data):
  N_doc = 0
  for i in range(len(Data)):
    if (Data[i] == 0):
      N_doc = N_doc + 1
  return N_doc

In [1203]:
def docu_count_positive_negative (Data, attribute):
  if attribute == "positive":
    C = docu_positive(Data)
  else:
    C = docu_negative(Data)
  return C


In [1204]:
P_doc = docu_count_positive_negative (y_Train, attribute= "positive")
N_doc = docu_count_positive_negative (y_Train, attribute= "negative")


In [1205]:
print(P_doc)
print(N_doc)

387
373


In [1206]:
def Number_Words_in_Doc(W, Count, attribute):
  Count1 = 0
  if attribute == "positive":
    if W in P_Words.keys():
      Count1 = P_Words[W]
    else:
      Count1 = 0

  elif attribute == "negative":
    if W in N_Words.keys():
      Count1 = N_Words[W]
    else:
      Count1 = 0
  return Count1 
  

In [1207]:
n = Number_Words_in_Doc('the', Count, attribute= "positive")
m = n / P_doc
print("p[the|positive]: " +str(m*100)+"%")

p[the|positive]: 34.62532299741602%


# **D) Calulating Accuracy using the Development Dataset**

In [1208]:
#probability for p[positive|the] without NAIVE BAYES
probability1 = (m*(P_doc/len(x_Train)))/ The_Probability
print("Probability of given word in data",probability1*100)  

Probability of given word in data 44.96644295302013


In [1209]:
def Naive_Bayes (Data, label,W, Count, attribute):
  n = Number_Words_in_Doc(W, Count, attribute)
  D = Probability_of_the_in_Dataset(Data, W)
  PN = docu_count_positive_negaive(label,attribute)
  if D == 0:
    D = 0.1
  else:
    D = D
  Result = (n/PN)*(PN/len(Data))/D
  return Result

In [1210]:
def Naive_Bayes_Data (Data, label, Count, data1):
  Prediction_P = 1
  Prediction_N = 1

  pred = []
  for index, Reviews in enumerate(data1):
    for W in Reviews.split(" "):
      NB = Naive_Bayes(Data, label, W, Count, attribute= "negative")
      if NB == 0:
        NB = 0.1
      else:
        NB = NB 
      
      Prediction_N = Prediction_N * NB 


      NB1 = Naive_Bayes(Data, label, W, Count, attribute= "positive")
      if NB1 == 0:
        NB1 = 0.1
      else:
        NB1 = NB1
      
      Prediction_P = Prediction_P * NB1 

    if Prediction_P > Prediction_N:
      pred.append(1)
    else:
      pred.append(0)

  return pred

In [1211]:
pred = Naive_Bayes_Data(x_Train,y_Train,Count,x_Dev)

In [1212]:
def Accuracy(label, Prediction):
  pred1 = 0
  for i in range(len(label)):
    if(label[i]== Prediction[i]):
      pred1 = pred1 + 1
    Z = float(pred1/len(label))
  return Z


In [1213]:
Accuracy1 = Accuracy(y_Dev, pred)
print("Accuracy Without Smoothing Data: "+str(Accuracy1*100)+"%")

Accuracy Without Smoothing Data: 52.5%


# **E) Comparing the effect of Smoothing**

# DATA Smoothing

In [1214]:
def Naive_Bayes_Smooth(Data, label, W,Count, attribute):
    n = Number_Words_in_Doc(W, Count, attribute)
    D = Probability_of_the_in_Dataset(Data, W)
    PN = docu_count_positive_negaive(label,attribute)
    if D == 0:
        D = 0.1
    else:
        D = D
    Result = (((n/PN)*(PN/len(Data)))+1)/(D+2)
    return Result
def Naive_Bayes_for_Smoothing_data(Data,label,Count,data1):
    Prediction_P = 1
    Prediction_N = 1
    pred = []
    for index,Reviews in enumerate(data1):
        for W in Reviews.split(" "):
            NB = Naive_Bayes_Smooth(Data, label, W,Count, attribute = "negative")
            if NB == 0:
                NB = 0.1
            else:
                NB = NB 
            Prediction_N = Prediction_N * NB 
            #print("n",negative_pred)
            NB1 = Naive_Bayes_Smooth(Data, label, W,Count, attribute = "positive")
            if NB1 == 0:
                NB1 = 0.1
            else:
                NB1 = NB1
            Prediction_P = Prediction_P * NB1
            #print("p",positive_pred)
        if Prediction_P > Prediction_N:
            pred.append(1)
        else:
            pred.append(0)
    return pred

In [1215]:
pred = Naive_Bayes_for_Smoothing_data(x_Train,y_Train, Count, x_Dev)

In [1216]:
Accuracy1 = Accuracy(y_Dev, pred)
print("Accuracy Without Smoothing Data: "+str(Accuracy1*100)+"%")

Accuracy Without Smoothing Data: 53.75%


# **Derive Top ten words that predicts Positive and Negative class**

In [1217]:
import operator, itertools
Positive_top = {}
for W in Count.keys():
  Z = Naive_Bayes(x_Train, y_Train, W, Count, attribute= "positive")
  Positive_top[W] = Z

top_reversed = dict(sorted(Positive_top.items(), key= operator.itemgetter(1),reverse=True))
Top_ten_Positive = dict(itertools.islice(top_reversed.items(), 10))

In [1218]:
print("Top Ten Words that Predict the Positive :",Top_ten_Positive.keys())

Top Ten Words that Predict the Positive : dict_keys(['Great', 'excellent', 'great', 'nice', 'best', 'Best', 'quite', 'menu', 'Vegas', 'fresh'])


In [1219]:
Negative_top = {}
for W in Count.keys():
  Z = Naive_Bayes(x_Train, y_Train, W, Count, attribute= "negative")
  Negative_top[W] = Z

top_reversed = dict(sorted(Negative_top.items(), key= operator.itemgetter(1),reverse=True))
Top_ten_Negative = dict(itertools.islice(top_reversed.items(), 10))

In [1220]:
print("Top Ten Words that Predict the Negative :",Top_ten_Negative.keys())

Top Ten Words that Predict the Negative : dict_keys(["wasn't", 'bad', 'being', 'much', 'got', 'coming', 'getting', 'never', 'better', 'too'])


# **F) Using the Test Dataset and calulating the Final Accuracy**

In [1221]:
S = Data.sample(frac=1).to_numpy()


# **Shuffling the data to get the Random Values in Training set and Development set**

In [1222]:
size = int(0.24*len(Data))
Test = S[:size]
print("Size of the Test set is : ",len(Test))
print("Shape of the Test set is : ",{Test.shape})

Size of the Test set is :  240
Shape of the Test set is :  {(240, 2)}


In [1223]:
x_Test = Test[:,0]
y_Test = Test[:,-1]



## Accuracies Without and With Smoothing

# Accuracy Without Smoothing

In [1224]:
pred = Naive_Bayes_Data(x_Train, y_Train,Count, x_Test)

In [1225]:
Accuracy1 = Accuracy(y_Test,pred)
print("Accuracy of the Data without Smoothing :"+str(Accuracy1*100)+"%")

Accuracy of the Data without Smoothing :49.583333333333336%


# Accuracy after Smoothing

In [1226]:
pred = Naive_Bayes_for_Smoothing_data(x_Train, y_Train,Count, x_Test)



In [1227]:
Accuracy1 = Accuracy(y_Test,pred)
print("Accuracy of the Data with Smoothing :"+str(Accuracy1*100)+"%")


Accuracy of the Data with Smoothing :52.916666666666664%


# *** The Accuracy of the yelp Dataset is increased after applying the Smoothing Method***

# **References:**

https://chrisalbon.com/machine_learning/naive_bayes/naive_bayes_classifier_from_scratch/</br>
https://www.kaggle.com/omkarsabnis/sentiment-analysis-on-the-yelp-reviews-dataset</br>
https://pynative.com/python-random-shuffle/</br>
https://levelup.gitconnected.com/movie-review-sentiment-analysis-with-naive-bayes-machine-learning-from-scratch-part-v-7bb869391bab</br>
https://www.kaggle.com/athoul01/predicting-yelp-ratings-from-review-text</br>
https://urytrayudu1.medium.com/sentiment-analysis-for-yelp-review-classification-54b65c09ff7b</br>
